#### schedule library

- https://github.com/dbader/schedule
- https://schedule.readthedocs.io/en/stable/


FAQ:

Frequently Asked Questions

- How to execute jobs in parallel?
- How to continuously run the scheduler without blocking the main thread?
- Does schedule support timezones?
- What if my task throws an exception?
- How can I run a job only once?
- How can I cancel several jobs at once?
- I’m getting an AttributeError: 'module' object has no attribute 'every' when I try to use schedule. How can I fix this?


#### optimization
- https://github.com/JuliaOpt/NLopt.jl
- https://github.com/mlubin/JuMPChance.jl (probabilistic constraints)

In [1]:
import schedule
import time
import datetime

In [2]:
schedule.jobs

[]

In [3]:
def job():
    print("I'm working... time: ", datetime.datetime.now().hour,":",
          datetime.datetime.now().minute)

schedule.every(1).minutes.do(job)

Every 1 minute do job() (last run: [never], next run: 2017-09-27 11:58:13)

In [4]:
schedule.jobs

[Every 1 minute do job() (last run: [never], next run: 2017-09-27 11:58:13)]

In [5]:
while True:
    schedule.run_pending()
    time.sleep(1)

I'm working... time:  11 : 58
I'm working... time:  11 : 59
I'm working... time:  12 : 0
I'm working... time:  12 : 1


KeyboardInterrupt: 

## Making non sequencial job executions


By default, schedule executes all jobs serially. The reasoning behind this is that it would be difficult to find a model for parallel execution that makes everyone happy.

You can work around this restriction by running each of the jobs in its own thread:

```python
import threading
import time
import schedule


def job():
    print("I'm running on thread %s" % threading.current_thread())


def run_threaded(job_func):
    job_thread = threading.Thread(target=job_func)
    job_thread.start()


schedule.every(10).seconds.do(run_threaded, job)
schedule.every(10).seconds.do(run_threaded, job)
schedule.every(10).seconds.do(run_threaded, job)
schedule.every(10).seconds.do(run_threaded, job)
schedule.every(10).seconds.do(run_threaded, job)


while 1:
    schedule.run_pending()
    time.sleep(1)
```

If you want tighter control on the number of threads use a shared jobqueue and one or more worker threads:

```python
import Queue
import time
import threading
import schedule


def job():
    print("I'm working")


def worker_main():
    while 1:
        job_func = jobqueue.get()
        job_func()
        jobqueue.task_done()

jobqueue = Queue.Queue()

schedule.every(10).seconds.do(jobqueue.put, job)
schedule.every(10).seconds.do(jobqueue.put, job)
schedule.every(10).seconds.do(jobqueue.put, job)
schedule.every(10).seconds.do(jobqueue.put, job)
schedule.every(10).seconds.do(jobqueue.put, job)

worker_thread = threading.Thread(target=worker_main)
worker_thread.start()

while 1:
    schedule.run_pending()
    time.sleep(1)
```

This model also makes sense for a distributed application where the workers are separate processes that receive jobs from a distributed work queue. I like using beanstalkd with the beanstalkc Python library.